## The sole purpose of this notebook is to comprehend the addition of user profiles to the data set

In [38]:
import pandas as pd

In [143]:
user_profiles = pd.read_csv('data/tweets/TwitterUserClassification.csv')
df = pd.read_csv('data/tweets/IchBinHanna.csv')
#get the correct data
#df= df[['author.description', 'author.id', 'author.name', 'author.public_metrics.followers_count', 'author.public_metrics.following_count', 'author.username', 'author_id', 'created_at', 'id', 'lang', 'public_metrics.retweet_count','text', 'hashtags','reference_type','wanted_tag']]
df['new_date'] = pd.to_datetime(df['created_at']).dt.strftime('%Y-%m-%d %H:%M:%S')
df = df.loc[(df['new_date'] > '2021-06-01 00:00:00') & (df['new_date'] <= '2021-09-30 23:59:59')]
df= df[['author.description', 'author.id', 'author.name', 'author.public_metrics.followers_count', 'author.public_metrics.following_count', 'author.username', 'author_id', 'created_at', 'id', 'lang', 'public_metrics.retweet_count','text', 'hashtags','reference_type','wanted_tag']]
#df = df.loc[df['reference_type'] != 'retweeted']
#df = df[df['text'].str.contains("#ichbinhanna", case = False)]
df =df.loc[df['wanted_tag']== True]
df = df.loc[(df['lang'] == 'de') | (df['lang'] == 'en')]

C:\Users\Admin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (6,8,9,10,13,37,41,42,55) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [40]:
#total number of tweets covered by the user profiles
user_profiles['totaltweetsbyauthor'].sum()

99669

In [140]:
user_profiles

,author.id,author.username,author.name,author.description,user.group,totaltweetsbyauthor
0,1.718151e+08,thothiel,Thorsten Thiel,"Politikwissenschaft, Politische Theorie, Wiss...",Academic unspecified,25
1,7.189508e+17,jtetza,Arbeitsrecht,Jeden Tag ein Tweet zum Arbeitsrecht.\nPräsent...,NaN,4
2,2.609103e+09,yrlaNor,@yrlaNor@social.tchncs.de ☀️,wahrheit ist arbeit. @angstalt @C3S @RKWardNet...,NaN,21
3,3.024848e+09,hrnschrttmchr,Jan,hier privat,NaN,11
4,2.565771e+09,Internaturalbaw,Internaturals,Pop Culture|Nature|Travel \n\nhttps://t.co/hyi...,NaN,11
...,...,...,...,...,...,...
15747,1.248565e+18,hmheinig,@hmheinig,Öffentliches Recht & Kirchenrecht + dies & das...,NaN,1
15748,3.096281e+09,jkflss,Joke F.,cahiers in Zeiten von Social Media.,NaN,2
15749,1.372144e+09,marius_s_klug,Marius Klug,"PhD student at BeMoBIL of @KlausGramann, resea...",Promovierende,1
15750,1.428095e+18,Agustin69501103,Agustina Carrizo,Always looking for the right question,NaN,1


In [141]:
df

,author.description,author.id,author.name,author.public_metrics.followers_count,author.public_metrics.following_count,author.username,author_id,created_at,id,lang,public_metrics.retweet_count,text,hashtags,reference_type,wanted_tag
0,Anthropologist/Decolonialism/Provenance resear...,4.872233e+09,Diego Ballestero,736.0,996.0,diballestero,4.872233e+09,2021-07-26T21:40:18.000Z,1419774606198640648,en,4.0,"Summer break. Pause, leisure, relaxation. Mayb...",['IchbinHanna'],no_reference,True
1,"Lecturer in English Literature @kieluni , depu...",3.079579e+09,Dr Melissa Schuh,855.0,735.0,schuh_melissa,3.079579e+09,2021-07-26T23:37:48.000Z,1419804175605669892,en,4.0,"RT @diballestero: Summer break. Pause, leisure...",['IchbinHanna'],retweeted,True
3,Anthropologist/Decolonialism/Provenance resear...,4.872233e+09,Diego Ballestero,736.0,996.0,diballestero,4.872233e+09,2021-07-26T21:40:23.000Z,1419774623575678982,en,0.0,Hanna will be just another number in the acade...,"['WirsindHanna', 'IchbinHanna']",replied_to,True
4,"Systems Biology, Computational lipidomics/meta...",1.343668e+18,Tim Rose,56.0,164.0,TimTheRose,1.343668e+18,2021-07-26T17:00:00.000Z,1419704066167513089,de,4.0,Am Donnerstag moderiere ich eine Diskussionsru...,"['Freising', 'Hochschule', 'Wissenschaft', 'Ic...",no_reference,True
6,"antropology/religious studies/theology, qualit...",1.242873e+18,Evelyn Reuter,112.0,165.0,EvelynReuter,1.242873e+18,2021-07-26T12:20:11.000Z,1419633644952502274,de,3.0,Sehenswertes Interview mit konstruktiven Gedan...,"['KleineFächer', 'DVRW', 'AKMN', 'IchbinHanna']",quoted,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116922,+++ Communications Designer +++ Brand Develope...,8.837853e+17,Tom Greis,470.0,1127.0,tomgreis,8.837853e+17,2021-07-27T05:48:46.000Z,1419897532218494978,de,28.0,RT @mlewandowsky: Die Debatte um #IchbinHanna ...,['IchbinHanna'],retweeted,True
116923,"Historiker - und trotzdem Optimist, außerdem A...",3.295060e+08,Mirko Gründer,81.0,658.0,MirkoGruender,3.295060e+08,2021-07-27T05:04:21.000Z,1419886351265247253,de,28.0,RT @mlewandowsky: Die Debatte um #IchbinHanna ...,['IchbinHanna'],retweeted,True
116924,NaN,1.418801e+18,Trapezit,3.0,0.0,Kommsemit,1.418801e+18,2021-07-27T04:16:11.000Z,1419874233539235842,de,0.0,Ihr armen Hannas. Im 1.Semester hat man euch g...,['IchbinHanna'],no_reference,True
116925,Leveraging #ComputationalHumanities for #alche...,1.162662e+18,Sarah Lang,936.0,2083.0,SarahALang_,1.162662e+18,2021-07-27T01:29:03.000Z,1419832172089946122,en,14.0,"RT @jenniferhenkeHB: With some delay, here is ...","['IchBinHanna', 'Ichbinreyhan']",retweeted,True


In [41]:
#unique authors in IchBinHanna.csv
len(pd.unique(df['author.id']))

13431

In [110]:
len(user_profiles)

15752

In [42]:
#unique classified users
len(pd.unique(user_profiles['author.id']))

15649

In [112]:
user_merge = user_profiles[['author.id', 'user.group']].drop_duplicates(subset= 'author.id')

In [113]:
len(user_merge)

15649

In [125]:
print(user_merge['author.id'].value_counts())

1.718151e+08    1
1.150763e+18    1
9.937993e+17    1
9.686273e+17    1
1.660430e+07    1
               ..
1.108036e+18    1
4.066168e+07    1
1.295380e+18    1
9.810603e+17    1
1.286620e+07    1
Name: author.id, Length: 15649, dtype: int64


In [145]:
#add profile info to tweets
merged_df = df.merge(user_merge,how='left' )

In [146]:
#remaining tweets
len(merged_df)

59263

In [147]:
#initial tweets
len(df)

60769

In [73]:
#remaining unique users
#len(pd.unique(merged_df['author.id']))

In [129]:
#distribution of user profiles in combined data frame
test = merged_df['user.group'].value_counts()
test

Postdoc                 17798
Academic unspecified     9916
Promovierende            3597
Prof                     3005
Bot                      1425
Media                    1094
Student                   908
Institution               686
Union rep                 555
Political rep             364
Jun. prof                 318
Teacher                   135
Medical doctor              5
Name: user.group, dtype: int64

In [130]:
unique_merged = merged_df.drop_duplicates(subset='author.id', keep="last")

In [131]:
#distribution of unique users in combined data frame
unique_merged['user.group'].value_counts()

Academic unspecified    1967
Postdoc                 1168
Promovierende            775
Prof                     614
Media                    287
Student                  274
Political rep            109
Teacher                   78
Institution               57
Union rep                 51
Jun. prof                 32
Bot                       23
Medical doctor             3
Name: user.group, dtype: int64

In [132]:
#distribution in user profile data frame
user_profiles['user.group'].value_counts()

Academic unspecified    2392
Postdoc                 1376
Promovierende            955
Prof                     716
Student                  339
Media                    331
Political rep            132
Teacher                   96
Institution               76
Union rep                 70
Jun. prof                 38
Bot                       29
Medical doctor             3
Name: user.group, dtype: int64

In [148]:
#identify users that are unclassified
diff = pd.concat([df,merged_df]).drop_duplicates(subset= 'id',keep = False)

In [152]:
len(pd.unique(diff['author_id']))

393

In [150]:
for i in user_profiles['author.description'].loc[user_profiles['author.username'] == "Duonna_Antonia"]:
    print(i)

Schreibt – recherchiert – plaudert, am liebsten über Menschenrechte und Politik im 17. oder 21. Jahrhundert.\nBuch: Bergünerstein I. Der Krieg\n#Wähencontent
Schreibt – recherchiert – plaudert, am liebsten über Menschenrechte und Politik im 17. oder 21. Jahrhundert.\nBuch: Bergünerstein I. Der Krieg\n#Wähencontent


In [151]:
diff

,author.description,author.name,author.public_metrics.followers_count,author.public_metrics.following_count,author.username,author_id,created_at,id,lang,public_metrics.retweet_count,text,hashtags,reference_type,wanted_tag,author.id,user.group
22,Arbeitsstelle Kleine Fächer,kleinefaecher,614.0,133.0,kleinefaecher,8.090261e+17,2021-07-26T09:00:23.000Z,1419583364516958213,de,5.0,Zur Situation des wissenschaftlichen Nachwuchs...,['IchbinHanna'],no_reference,True,NaN,NaN
102,Post-Doc Researcher in PolComm at @univienna. ...,Jakob-Moritz Eberl,2811.0,1204.0,JaMoEberl,7.361777e+17,2021-07-26T15:39:58.000Z,1419683924708581380,de,27.0,RT @mlewandowsky: Die Debatte um #IchbinHanna ...,['IchbinHanna'],retweeted,True,NaN,NaN
248,Irgendwas mit Geschichte. Vor allem Frühe Neuz...,Tobias Winnerling,1167.0,960.0,t_winnerling,9.529962e+17,2021-07-26T09:31:50.000Z,1419591280590475264,de,3.0,RT @broxben: @NGA_Wiss Wenn #IchBinHanna sich ...,['IchBinHanna'],retweeted,True,NaN,NaN
317,Jacobin ist eine führende Publikation der sozi...,Jacobin Magazin,13479.0,300.0,jacobinmag_de,9.159715e+17,2021-06-11T11:43:13.000Z,1403316889049239559,de,29.0,Das Wissenschaftszeitvertragsgesetz prekarisie...,['IchbinHanna'],no_reference,True,NaN,NaN
344,Irgendwas mit Geschichte. Vor allem Frühe Neuz...,Tobias Winnerling,1167.0,960.0,t_winnerling,9.529962e+17,2021-07-25T21:28:07.000Z,1419409149008416779,de,6.0,"RT @dawwidd: Verschrottet, was euch verschrott...","['IchBinHanna', 'IchBinReyhan']",retweeted,True,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116506,Irgendwas mit Geschichte. Vor allem Frühe Neuz...,Tobias Winnerling,1300.0,1078.0,t_winnerling,9.529962e+17,2021-07-27T11:53:44.000Z,1419989378076655618,de,8.0,RT @JanCloppenburg: Nächster Akt von #IchbinHa...,['IchbinHanna'],retweeted,True,NaN,NaN
116768,Irgendwas mit Geschichte. Vor allem Frühe Neuz...,Tobias Winnerling,1300.0,1078.0,t_winnerling,9.529962e+17,2021-07-27T09:52:24.000Z,1419958844604694560,de,6.0,RT @T19R84: Selbst mit der Professur ist es ni...,['IchbinHanna'],retweeted,True,NaN,NaN
116867,Irgendwas mit Geschichte. Vor allem Frühe Neuz...,Tobias Winnerling,1300.0,1078.0,t_winnerling,9.529962e+17,2021-07-27T09:02:53.000Z,1419946382123835394,de,41.0,RT @AmreiBahr: Prekäre Arbeitsverhältnisse sin...,['IchBinHanna'],retweeted,True,NaN,NaN
116922,+++ Communications Designer +++ Brand Develope...,Tom Greis,470.0,1127.0,tomgreis,8.837853e+17,2021-07-27T05:48:46.000Z,1419897532218494978,de,28.0,RT @mlewandowsky: Die Debatte um #IchbinHanna ...,['IchbinHanna'],retweeted,True,NaN,NaN


In [153]:
user_profiles.loc[user_profiles['author.name'] == "Tom Greis"]

,author.id,author.username,author.name,author.description,user.group,totaltweetsbyauthor
12278,8.837853e+17,tomgreis,Tom Greis,+++ Communications Designer +++ Brand Develope...,NaN,2


In [156]:
df.loc[df['author.name'] == "Tom Greis"]

,author.description,author.name,author.public_metrics.followers_count,author.public_metrics.following_count,author.username,author_id,created_at,id,lang,public_metrics.retweet_count,text,hashtags,reference_type,wanted_tag
37294,+++ Communications Designer +++ Brand Develope...,Tom Greis,458.0,1098.0,tomgreis,8.837853e+17,2021-06-25T12:41:15.000Z,1408404922798948357,de,1090.0,RT @AchimLandwehr: Ich bin nicht Hanna. #Ichbi...,['IchbinHanna'],retweeted,True
116922,+++ Communications Designer +++ Brand Develope...,Tom Greis,470.0,1127.0,tomgreis,8.837853e+17,2021-07-27T05:48:46.000Z,1419897532218494978,de,28.0,RT @mlewandowsky: Die Debatte um #IchbinHanna ...,['IchbinHanna'],retweeted,True
